In [46]:
from pymol import cmd

# Mapping of one-letter to three-letter amino acid codes
amino_acid_map = {
    'A': 'ALA',  # Alanine
    'R': 'ARG',  # Arginine
    'N': 'ASN',  # Asparagine
    'D': 'ASP',  # Aspartic Acid
    'C': 'CYS',  # Cysteine
    'E': 'GLU',  # Glutamic Acid
    'Q': 'GLN',  # Glutamine
    'G': 'GLY',  # Glycine
    'H': 'HIS',  # Histidine
    'I': 'ILE',  # Isoleucine
    'L': 'LEU',  # Leucine
    'K': 'LYS',  # Lysine
    'M': 'MET',  # Methionine
    'F': 'PHE',  # Phenylalanine
    'P': 'PRO',  # Proline
    'S': 'SER',  # Serine
    'T': 'THR',  # Threonine
    'W': 'TRP',  # Tryptophan
    'Y': 'TYR',  # Tyrosine
    'V': 'VAL',  # Valine
}

# Function to mutate a residue in a given PDB file
def mutate_residues(pdb_file, mutations, output_dir="output_directory"):
    """
    Mutate residues in a PDB file using PyMOL.
    
    Parameters:
    - pdb_file: Path to the input PDB file.
    - mutations: List of mutation tuples, e.g., [(4, "S", "R")].
      Each tuple should be in the form (residue_number, original_one_letter, mutated_one_letter).
    - output_dir: Directory to save the modified PDB file.
    """
    # Load the PDB file
    cmd.load(pdb_file, "structure")
    
    # Loop through each mutation request
    for resi, original_residue, target_residue in mutations:
        # Mutation will always apply to chain X in the structure
        selection = f"chain X and resi {resi}"
        
        # Apply mutation using PyMOL's mutagenesis wizard
        cmd.wizard("mutagenesis")
        cmd.do(f"select {selection}")
        cmd.get_wizard().do_select(selection)
        
        # Set the target residue for mutation using three-letter code
        if target_residue == "H":  # Check if the target residue is Histidine
            three_letter_residue = "HID"  # Use HID for the imidazole form of histidine
        else:
            three_letter_residue = amino_acid_map[target_residue]
        
        cmd.get_wizard().set_mode(three_letter_residue)
        cmd.get_wizard().apply()
        
        # Use alter command to ensure correct naming with full amino acid names
        cmd.alter(selection, f"resn='{three_letter_residue}'")  # Set residue name to full amino acid name
        cmd.alter(selection, "name=name")  # Reassign correct atom names
        
        # Close the mutagenesis wizard
        cmd.set_wizard()

    # Remove hydrogens after mutations
    cmd.remove("hydro")
    
    # Generate output file name using original and mutated one-letter codes
    output_file = f"{output_dir}/{original_residue}{resi}{target_residue}.pdb"
    
    # Save the mutated structure to a new PDB file
    cmd.save(output_file, "structure", format="pdb")
    
    # Clear all after processing
    cmd.delete("all")
    print(f"Mutations applied. Saved to {output_file}.")

# Usage example
pdb_file_path = "/home/hp/nayanika/github/GPX6/prep_structures/original_mousecys.pdb"
mutations_list = [
    (195, "S", "N"),   # This will apply mutation to chain X
    # Add more mutations as needed
]
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/mousecys"

# Run the mutation function
mutate_residues(pdb_file_path, mutations_list, output_dir)


PyMOL>select chain X and resi 195
Selected!
 ExecutiveRMSPairs: RMSD =    0.036 (4 to 4 atoms)
 Mutagenesis: no phi/psi, using backbone-independent rotamers.
 Mutagenesis: 6 rotamers loaded.
 Rotamer 2/6, strain=16.56
 ExecutiveRMSPairs: RMSD =    0.038 (4 to 4 atoms)
 Mutagenesis: no phi/psi, using backbone-independent rotamers.
 Mutagenesis: 18 rotamers loaded.
 Rotamer 10/18, strain=16.33
Mutations applied. Saved to /home/hp/nayanika/github/GPX6/prep_structures/mousecys/S195N.pdb.
